In [3]:
################################
#Packages
using Gurobi
using JuMP
using Plots
using GLMNet
using DataFrames 
using CSV
using Statistics
using Random


In [39]:
impute_filepath = "Data/data_preimpute.csv"

xtrain = CSV.read(impute_filepath,DataFrame)

allowmissing!(xtrain)
names(xtrain)

35-element Vector{String}:
 "Item_Weight"
 "Item_Visibility"
 "Item_MRP"
 "Item_Outlet_Sales"
 "Years_Opened"
 "Item_Type_Baking Goods"
 "Item_Type_Breads"
 "Item_Type_Breakfast"
 "Item_Type_Canned"
 "Item_Type_Dairy"
 "Item_Type_Frozen Foods"
 "Item_Type_Fruits and Vegetables"
 "Item_Type_Hard Drinks"
 ⋮
 "Outlet_Identifier_OUT018"
 "Outlet_Identifier_OUT019"
 "Outlet_Identifier_OUT027"
 "Outlet_Identifier_OUT035"
 "Outlet_Identifier_OUT045"
 "Outlet_Identifier_OUT046"
 "Outlet_Identifier_OUT049"
 "Outlet_Location_Type_Tier 2"
 "Outlet_Location_Type_Tier 3"
 "Outlet_Type_Supermarket Type1"
 "Outlet_Type_Supermarket Type2"
 "Outlet_Type_Supermarket Type3"

In [40]:
for i = 1:nrow(xtrain)
    if xtrain[i, :Item_Visibility] == 9999
        xtrain[i,:Item_Visibility] = missing
        
    end
    
    if xtrain[i, :Item_Weight] == 9999
        xtrain[i,:Item_Weight] = missing
        
    end
    
#     if xtrain[i, :Outlet_Size] == "missing"
#         xtrain[i,:Outlet_Size] = missing
        
#     end
    
end

In [41]:
xtrain[:,1:6]

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Years_Opened,Item_Type_Baking Goods
,Float64?,Float64?,Float64?,Float64?,Int64?,Int64?
1,9.3,0.0160473,249.809,3735.14,14,0
2,5.92,0.0192782,48.2692,443.423,4,0
3,17.5,0.0167601,141.618,2097.27,14,0
4,19.2,missing,182.095,732.38,15,0
5,8.93,missing,53.8614,994.705,26,0
6,10.395,missing,51.4008,556.609,4,1
7,13.65,0.0127411,57.6588,343.553,26,0
8,missing,0.12747,107.762,4022.76,28,0
9,16.2,0.0166871,96.9726,1076.6,11,0


In [42]:
X_imputed = IAI.impute(xtrain)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/iai/builds/InterpretableAI/SystemImage/SysImgBuilder/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Trying different warm starts...    100%|████████████████| Time: 0:08:36
4m  Warmstart:  rand



,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Years_Opened,Item_Type_Baking Goods
,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,9.3,0.0160473,249.809,3735.14,14.0,0.0
2,5.92,0.0192782,48.2692,443.423,4.0,0.0
3,17.5,0.0167601,141.618,2097.27,14.0,0.0
4,19.2,0.0698715,182.095,732.38,15.0,0.0
5,8.93,0.0585074,53.8614,994.705,26.0,0.0
6,10.395,0.0589066,51.4008,556.609,4.0,1.0
7,13.65,0.0127411,57.6588,343.553,26.0,0.0
8,12.0914,0.12747,107.762,4022.76,28.0,0.0
9,16.2,0.0166871,96.9726,1076.6,11.0,0.0


In [53]:
names(xtrain)[20:35]

16-element Vector{String}:
 "Item_Type_Starchy Foods"
 "Item_Fat_Content_Regular"
 "Outlet_Identifier_OUT013"
 "Outlet_Identifier_OUT017"
 "Outlet_Identifier_OUT018"
 "Outlet_Identifier_OUT019"
 "Outlet_Identifier_OUT027"
 "Outlet_Identifier_OUT035"
 "Outlet_Identifier_OUT045"
 "Outlet_Identifier_OUT046"
 "Outlet_Identifier_OUT049"
 "Outlet_Location_Type_Tier 2"
 "Outlet_Location_Type_Tier 3"
 "Outlet_Type_Supermarket Type1"
 "Outlet_Type_Supermarket Type2"
 "Outlet_Type_Supermarket Type3"

In [56]:
store_13_idx = findall(x->x ==1, xtrain[:, :Outlet_Identifier_OUT013]);
store_13 = X_imputed[store_13_idx,:]
store_13_Vis = store_13[:,:Item_Visibility]
sum(store_13_Vis)

98.8357406869001

In [64]:
store_13_idx = findall(x->x ==1, xtrain[:, :Outlet_Identifier_OUT046]);
store_13 = X_imputed[store_13_idx,:]
store_13_Vis = store_13[:,:Item_Visibility]
sum(store_13_Vis)

98.57893871147326

In [67]:
CSV.write("Data/Optimal_Impute_Data.csv", X_imputed)

"Data/Optimal_Impute_Data.csv"